In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_2687/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
from ROOT import *
import time, os, math, sys, json, gc, copy, pprint, argparse

from array import array

#import histoHelpers
#from histoHelpers import *

from random import gauss
import numpy as np

#sys.path.insert(0,'../../')
#from datasets_dijetSel_RunIISummer20UL_SampleDictPrep import dictSamples, checkDict

#sys.path.insert(0,'../python/')
#import CMS_lumi as CMS_lumi
#import tdrstyle as tdrstyle
from collections import OrderedDict
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
sys.path.insert(0,'../python/')
from DoubleCrystalBall import Base, Distribution, DLog, Integral

fitter = ROOT.Fit.Fitter()
myDist = Distribution()
myInt = Integral()
myDLog = DLog()
#params = [N,mu,sigma,kL,nL,,kR,nR]

params = array('f', [1,400,50,0,1,500,2])
myDist.SetParameters(params)
myInt.SetParameters(params)
myDLog.SetParameters(params)
myDist.dump()
myInt.dump()
myDLog.dump()

x_values = array('f', [i for i in np.linspace(100.,1000., 900)]) 
resultsD = []
resultsI = []
resultsDL = []

for x in x_values:
    resultD = myDist.DoEval([x])  #  [x] because Eval expects array-like input
    resultsD.append(resultD)
    
    resultDL = myDLog.DoEval([x])  
    resultsDL.append(resultDL)
    
    resultI = myInt.DoEval([x])  
    resultsI.append(resultI)

plt.figure(figsize=(12,9))
plt.plot(x_values,resultsD,label='Double Crystal-Ball')
plt.plot(x_values,resultsI,label='Integral')
plt.plot(x_values,resultsDL,label='Derivative of logarithm')
#plt.plot(x_values,resultsD)
plt.legend(loc='best', fontsize=12 )
plt.show()
myDist.dump()
myInt.dump()
print(resultsI)


In [ ]:
import ROOT
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import numpy as np

# Assume your TH1 histogram is named h_eff
h_eff = ROOT.TH1F("h_eff", "Efficiency", 100, 0, 500)  # Example histogram, you'd use your actual histogram here

# Convert TH1 to Python lists
x_vals = [h_eff.GetBinCenter(i) for i in range(1, h_eff.GetNbinsX()+1)]
y_vals = [h_eff.GetBinContent(i) for i in range(1, h_eff.GetNbinsX()+1)]
y_errs = [h_eff.GetBinError(i) for i in range(1, h_eff.GetNbinsX()+1)]

# Define the objective function for hyperopt
def objective(params):
    cb = Distribution()  # From your implementation
    f_cb = ROOT.TF1("f_cb", cb.wrapped_eval, 0, 500, 7)  # Adjust the range accordingly
    
    for i, p in enumerate(params):
        f_cb.SetParameter(i, p)
        f_cb.SetParLimits(i, params[p][0], params[p][1])
    
    h_eff.Fit(f_cb, "R")  # R option is for the range
    
    chi2_ndf = f_cb.GetChisquare() / f_cb.GetNDF()
    
    return {'loss': chi2_ndf, 'status': STATUS_OK, 'params': params}

# Define the parameter space for hyperopt
space = {
    'N': (0, 2),
    'mu': (380, 460),  # Sample range, adjust accordingly
    'sigma': (0, 10),
    'kL': (-5, 5),
    'nL': (0, 5),
    'kR': (-5, 5),
    'nR': (0, 5)
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

# Extract the best 5 fits and display them
best_trials = sorted(trials.results, key=lambda x: x['loss'])[:5]
canvas = ROOT.TCanvas("canvas", "Best Fits", 800, 600)
h_eff.Draw()

leg = ROOT.TLegend(0.6, 0.6, 0.9, 0.9)
for i, trial in enumerate(best_trials):
    params = trial['params']
    cb = Distribution()
    f_cb = ROOT.TF1(f"f_cb_{i}", cb.wrapped_eval, 0, 500, 7)
    for j, p in enumerate(params):
        f_cb.SetParameter(j, params[p])
    f_cb.SetLineColor(i+1)  # Different color for each fit
    f_cb.Draw("SAME")
    leg.AddEntry(f_cb, f"Fit {i+1}: chi2/ndf={trial['loss']:.2f}", "l")

leg.Draw()
canvas.Print("best_fits.png")

# Determine where efficiency is >= 99.5%
efficiency_threshold = 0.995
thresholds = []
for x in x_vals:
    if f_cb.Eval(x) >= efficiency_threshold:
        thresholds.append(x)
print("Efficiency >= 99.5% at:", thresholds)